In [1]:
# download from: https://criteostorage.blob.core.windows.net/criteo-research-datasets/kaggle-display-advertising-challenge-dataset.tar.gz

import pyspark
from pyspark import SparkConf

In [2]:
dataset = ('hdfs://namenode/dataset/demo/readme.txt', 'hdfs://namenode/dataset/demo/train.txt', 'hdfs://namenode/dataset/demo/test.txt')

In [3]:
spark = pyspark.sql.SparkSession.builder.master("spark://datascience-spark:7077")\
    .config("spark.pyspark.python", "/opt/bitnami/python/bin/python3")\
    .config("spark.executor.memory", "4g")\
    .config("spark.driver.memory", "4g")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/06/22 16:34:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# from pyspark import SparkConf, SparkContext

# conf = SparkConf()

# conf.setAppName('metaspore-demo')
# conf.setMaster('spark://datascience-spark:7077')
# sc = SparkContext(conf=conf)

In [5]:
readme  = spark.read.text(dataset[0])
for line in readme.collect():
    print(line[0])

AnalysisException: Path does not exist: hdfs://namenode/dataset/demo/readme.txt

In [6]:
df_test = spark.read.csv(dataset[2], header=False, sep='\t')

In [7]:
#df_test.printSchema()

In [8]:
#df_test.show()

In [9]:
df_train = spark.read.csv(dataset[1], header=False, sep='\t')

In [10]:
#df_train.show()
print(df_train.count())

45840617


In [11]:
ROOT_DIR = '/home/dev/workspace/machine-learning/playground/datascience/data'
# ROOT_DIR = './data'

In [12]:
!mkdir data/data
!pwd

/home/dev/workspace/machine-learning/playground/datascience


In [13]:
import torch
import metaspore as ms

class DemoModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self._embedding_size = 16
        self._schema_dir = ROOT_DIR + '/schema/'
        self._column_name_path = self._schema_dir + 'column_name_demo.txt'
        self._combine_schema_path = self._schema_dir + 'combine_schema_demo.txt'
        self._sparse = ms.EmbeddingSumConcat(self._embedding_size, self._column_name_path, self._combine_schema_path)
        self._sparse.updater = ms.FTRLTensorUpdater()
        self._sparse.initializer = ms.NormalTensorInitializer(var=0.01)
        self._dense = torch.nn.Sequential(
            ms.nn.Normalization(self._sparse.feature_count * self._embedding_size),
            torch.nn.Linear(self._sparse.feature_count * self._embedding_size, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1),
        )

    def forward(self, x):
        x = self._sparse(x)
        x = self._dense(x)
        return torch.sigmoid(x)

In [14]:
module = DemoModule()

loaded combine schema from combine schema file '/home/dev/workspace/machine-learning/playground/datascience/data/schema/combine_schema_demo.txt'
[2025-01-19 15:57:32.263] [info] [local_filesys.cpp:116] Opening local file /home/dev/workspace/machine-learning/playground/datascience/data/schema/combine_schema_demo.txt with mode r
integer_feature_1
[2025-01-19 15:57:32.264] [info] add expr bkdr_hash(integer_feature_1, StringBKDRHashFunctionOption::name=integer_feature_1)
integer_feature_2
[2025-01-19 15:57:32.264] [info] add expr bkdr_hash(integer_feature_2, StringBKDRHashFunctionOption::name=integer_feature_2)
integer_feature_3
[2025-01-19 15:57:32.264] [info] add expr bkdr_hash(integer_feature_3, StringBKDRHashFunctionOption::name=integer_feature_3)
integer_feature_4
[2025-01-19 15:57:32.264] [info] add expr bkdr_hash(integer_feature_4, StringBKDRHashFunctionOption::name=integer_feature_4)
integer_feature_5
[2025-01-19 15:57:32.264] [info] add expr bkdr_hash(integer_feature_5, StringBKDR

In [15]:
import torch

In [16]:
train_ds, val_ds = df_train.randomSplit(weights=[0.8, 0.2])

In [17]:
print(train_ds.count(), val_ds.count())

25/01/19 15:57:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


36680356 9160261


In [18]:
model_out_path = ROOT_DIR + '/output/dev/model_out/'
estimator = ms.PyTorchEstimator(module=module,
                                worker_count=1,
                                server_count=1,
                                model_out_path=model_out_path,
                                experiment_name='0.1',
                                input_label_column_index=0)

In [19]:
column_names = []
with open(f'{ROOT_DIR}/schema/column_name_demo.txt', 'r') as f:
    for line in f:
        column_names.append(line.split(' ')[1].strip())
print(column_names)

['label', 'integer_feature_1', 'integer_feature_2', 'integer_feature_3', 'integer_feature_4', 'integer_feature_5', 'integer_feature_6', 'integer_feature_7', 'integer_feature_8', 'integer_feature_9', 'integer_feature_10', 'integer_feature_11', 'integer_feature_12', 'integer_feature_13', 'categorical_feature_1', 'categorical_feature_2', 'categorical_feature_3', 'categorical_feature_4', 'categorical_feature_5', 'categorical_feature_6', 'categorical_feature_7', 'categorical_feature_8', 'categorical_feature_9', 'categorical_feature_10', 'categorical_feature_11', 'categorical_feature_12', 'categorical_feature_13', 'categorical_feature_14', 'categorical_feature_15', 'categorical_feature_16', 'categorical_feature_17', 'categorical_feature_18', 'categorical_feature_19', 'categorical_feature_20', 'categorical_feature_21', 'categorical_feature_22', 'categorical_feature_23', 'categorical_feature_24', 'categorical_feature_25', 'categorical_feature_26']


In [21]:
model = estimator.fit(train_ds)

[2025-01-19 15:58:58.512] [info] PS job with coordinator address 172.50.0.4:38171 started.
[2025-01-19 15:58:58.513] [info] PSRunner::RunPS: pid: 7552, tid: 8021, thread: 0x7f2a0062c700
[2025-01-19 15:58:58.513] [info] PSRunner::RunPSCoordinator: pid: 7552, tid: 8021, thread: 0x7f2a0062c700
[2025-01-19 15:58:58.515] [info] ActorProcess::Receiving: Coordinator pid: 7552, tid: 8026, thread: 0x7f29f87e8700
25/01/19 15:58:58 WARN TaskSetManager: Lost task 0.0 in stage 14.0 (TID 260) (172.50.0.18 executor 1): java.io.IOException: Cannot run program "/usr/bin/python3": error=2, No such file or directory
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1143)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1073)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:216)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:134)
	at org.apache.spark.api.python.PythonWorkerFactory.cre

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Could not recover from a failed barrier ResultStage. Most recent failure reason: Stage failed because barrier task ResultTask(14, 0) finished unsuccessfully.
java.io.IOException: Cannot run program "/usr/bin/python3": error=2, No such file or directory
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1143)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1073)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:216)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:134)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala

In [1]:
result = model.transform(val_ds)

NameError: name 'model' is not defined

In [2]:
result.show(5)

NameError: name 'result' is not defined

In [4]:
from pyspark.ml import evaluation
evaluator = evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(result)
print('test_auc: %g' % test_auc)

AssertionError: 